In [1]:
import yfinance as yf
import numpy as np
import datetime as dt
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
ticker=yf.Ticker('COST')
expiry=ticker.options[1]
print(f"Ticker: {ticker.ticker}")
print(f"Expiry: {expiry}")
options=ticker.option_chain(expiry).calls
prices=ticker.history(period="2y")['Close']
last_price=prices.iloc[-1]
print(f"Last price: {last_price}")
margin=3
options=options[(options['strike']>=last_price-margin)&(options['strike']<=last_price+margin)]
options

Ticker: COST
Expiry: 2025-10-31
Last price: 942.0499877929688


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
41,COST251031C00940000,2025-10-23 19:59:20+00:00,940.0,12.53,11.95,13.00,-5.46,-30.350195,73.0,353,0.214943,True,REGULAR,USD
42,COST251031C00942500,2025-10-23 19:43:23+00:00,942.5,11.10,10.75,11.45,-4.54,-29.028131,72.0,111,0.209755,False,REGULAR,USD
43,COST251031C00945000,2025-10-23 19:52:45+00:00,945.0,9.50,9.40,10.50,-2.63,-21.681780,226.0,228,0.213845,False,REGULAR,USD


In [ ]:
from scipy.optimize import brentq

r = (1+0.071)**(1/255)-1
S_0 = last_price
mu = prices.pct_change().mean()
t = (dt.datetime.strptime(expiry, "%Y-%m-%d") - dt.datetime.today()).days + 1
print(f"S_0:{S_0}")
print(f"mu:{mu}")
print(f"dt:{t}")
print(f"Expiry:{expiry}")
for K, l in zip(options['strike'], options['lastPrice']):
    def BS(sigma):
        d1 = (np.log(S0/K) + (r + sigma**2/2)*t) / (sigma*np.sqrt(t))
        d2 = d1 - sigma*np.sqrt(t)
        c = S0*stats.norm.cdf(d1) - K*np.exp(-r*t)*stats.norm.cdf(d2)
        return c

    def objective(sigma):
        return BS(sigma) - l

    implied_vol = brentq(objective, -0.01, 0.07, xtol=1e-6)

    print(f"La volatilidad implícita para strike en {K} es: {implied_vol:.6f}")

S_0:942.0499877929688
mu:0.001221242205829166
dt:7
Expiry:2025-10-31
La volatilidad implícita para strike en 940.0 es: 0.010592
La volatilidad implícita para strike en 942.5 es: 0.010492
La volatilidad implícita para strike en 945.0 es: 0.010127


### Después del vencimiento

In [8]:
prices = yf.download("COST", start="2025-10-24", end="2025-11-01")["Close"]


/tmp/ipython-input-877217295.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  prices = yf.download("COST", start="2025-10-24", end="2025-11-01")["Close"]
[*********************100%***********************]  1 of 1 completed


In [9]:
# Calcular retornos logarítmicos
returns = np.log(prices / prices.shift(1)).dropna()

# Calcular volatilidad realizada anualizada
realized_vol = float(np.sqrt(np.sum(returns**2) * (252 / len(returns))))

print(f"Volatilidad realizada anualizada: {realized_vol:.6f}")

Volatilidad realizada anualizada: 0.131828


/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/tmp/ipython-input-2238672557.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  realized_vol = float(np.sqrt(np.sum(returns**2) * (252 / len(returns))))


In [10]:
data = {
    "Fecha de cálculo": ["2025-10-24", "2025-10-24", "2025-10-24"],
    "Expiry": ["2025-10-31", "2025-10-31", "2025-10-31"],
    "Strike": [940, 942.5, 945],
    "Implied Volatility": [0.010592, 0.010492, 0.010127],
    "Anualizada": [0.168142787, 0.166555337, 0.160761141],
    "Last Price": [942.0499, 942.0499, 942.0499],
    "Vol realizada anualizada": [0.131828, 0.131828, 0.131828],
    "Diferencia absoluta": [0.036314787, 0.034727337, 0.028933141],
    "Error porcentual": [27.54709722, 26.3429139, 21.94764478]
}

df = pd.DataFrame(data)
df

,Fecha de cálculo,Expiry,Strike,Implied Volatility,Anualizada,Last Price,Vol realizada anualizada,Diferencia absoluta,Error porcentual
0,2025-10-24,2025-10-31,940.0,0.010592,0.168143,942.0499,0.131828,0.036315,27.547097
1,2025-10-24,2025-10-31,942.5,0.010492,0.166555,942.0499,0.131828,0.034727,26.342914
2,2025-10-24,2025-10-31,945.0,0.010127,0.160761,942.0499,0.131828,0.028933,21.947645


Al comparar la **volatilidad implícita** del 24 de octubre (alrededor del **16-17 % anualizado**) con la **volatilidad realizada** hasta el 31 de octubre (≈ 13 %), se observa que el mercado **sobreestimó ligeramente** la volatilidad futura del activo.

La diferencia fue de unos **3 puntos porcentuales** (entre un **22 % y 28 %** más alta).

Esto es común, ya que la volatilidad implícita suele incluir una **prima por riesgo**, reflejando la cautela del mercado ante posibles movimientos inesperados.
